Модель прогнозирования стоимости жилья для агентства недвижимости
=========================================================================================================

В этом ноутбуке представлена разработка модели для предсказания стоимости домов на основе истории предложений. Основная задача проекта – помочь агентству недвижимости обойти конкурентов по скорости и качеству проведения сделок за счёт автоматизации анализа объектов.

**Проблематика:**  
Риелторам агентства требуется слишком много времени на сортировку объявлений и поиск наиболее выгодных предложений, что негативно сказывается на финансовых показателях.

**Цели и задачи проекта:**  
- Провести детальный анализ данных (EDA) с использованием статистических тестов и визуализации.  
- Провести предварительную обработку и очистку данных для повышения качества модели.  
- Разработать и протестировать модели прогнозирования стоимости жилья.  
- Разработать сервис для интеграции модели в рабочий процесс, обеспечивающий быстрые и точные предсказания.

Давайте перейдём к анализу данных и дальнейшей разработке решения!

## Описание данных:

| Название столбца               | Содержание информации                                                                                         |
|--------------------------------|---------------------------------------------------------------------------------------------------------------|
| status                         | Статус продажи                                                                                                |
| private pool / PrivatePool     | Наличие собственного бассейна                                                                                 |
| propertyType                   | Тип объекта недвижимости                                                                                      |
| street                         | Адрес объекта                                                                                                 |
| baths                          | Количество ванных комнат                                                                                        |
| homeFacts                      | Сведения о строительстве объекта (несколько типов сведений, влияющих на оценку объекта)                         |
| fireplace                      | Наличие камина                                                                                                |
| city                           | Город                                                                                                         |
| schools                        | Сведения о школах в районе                                                                                      |
| sqft                           | Площадь в футах                                                                                               |
| zipcode                        | Почтовый индекс                                                                                               |
| beds                           | Количество спален                                                                                             |
| state                          | Штат                                                                                                          |
| stories                        | Количество этажей                                                                                             |
| mls-id / MlsId                 | Идентификатор MLS (Multiple Listing Service, система мультилистинга)                                            |
| target                         | Цена объекта недвижимости (целевой признак, который необходимо спрогнозировать)                               |

In [1]:
# Загрузка необходимых библиотек
import numpy as np
import pandas as pd
import time
import re
import ast

C:\Users\Koop_\AppData\Local\Temp\ipykernel_16472\1118506453.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Предобратка данных

## Первичный осмотр данных

Посмотрим на данные

In [2]:
data = pd.read_csv('data/data.csv')
display(data.head())
data.info()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103114 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226469 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

Почистим дубликаты в данных

In [3]:
# сохраняем количество строк до удаления дубликатов
rows_before = data.shape[0]
# удалим дубликаты
data = data.drop_duplicates(ignore_index=True)
# сохраняем количество строк после удаления дубликатов
rows_after = data.shape[0]
# считаем количество удаленных дубликатов
duplicates_removed = rows_before - rows_after
print(f'Удалено {duplicates_removed} дубликатов')

Удалено 50 дубликатов


## Посмотрим на пропуски в данных

In [4]:
# Вычисляем процент пропущенных значений для каждого столбца
missing_percentages = data.isnull().mean() * 100
# Выводим процент пропусков через цикл
print("Процент пропущенных значений по столбцам:")
for column, percent in missing_percentages.items():
    print(f"Столбец '{column:13}': {percent:6.2f}% пропусков")

Процент пропущенных значений по столбцам:
Столбец 'status       ':  10.58% пропусков
Столбец 'private pool ':  98.89% пропусков
Столбец 'propertyType ':   9.21% пропусков
Столбец 'street       ':   0.00% пропусков
Столбец 'baths        ':  28.19% пропусков
Столбец 'homeFacts    ':   0.00% пропусков
Столбец 'fireplace    ':  72.66% пропусков
Столбец 'city         ':   0.01% пропусков
Столбец 'schools      ':   0.00% пропусков
Столбец 'sqft         ':  10.75% пропусков
Столбец 'zipcode      ':   0.00% пропусков
Столбец 'beds         ':  24.20% пропусков
Столбец 'state        ':   0.00% пропусков
Столбец 'stories      ':  39.95% пропусков
Столбец 'mls-id       ':  93.39% пропусков
Столбец 'PrivatePool  ':  89.31% пропусков
Столбец 'MlsId        ':  17.73% пропусков
Столбец 'target       ':   0.66% пропусков


In [5]:
# посмотрим что содержится в private pool и PrivatePool
display(data['private pool'].unique())
display(data['PrivatePool'].unique())

array([nan, 'Yes'], dtype=object)

array([nan, 'yes', 'Yes'], dtype=object)

Что можно сказать по пропускам в данных:

- Столбцы «private pool» (98.89%) и «PrivatePool» (89.31%) практически не заполнены, содержат дублирующую информацию,Предполагаем, что если информация отсутствует, то это означает отсутствие бассейна.
- Столбцы 'mls-id' и 'MlsId', поскольку они используются для идентификации объектов и не влияют на модель прогнозирования.
- Столбцы «fireplace» (72.66%), «baths» (28.19%) и «stories» (39.95%) также имеют заметный уровень пропусков, что может повлиять на качество модели и потребует продуманного подхода к обработке: заполнение, удаление или специальное кодирование пропусков.
- Некоторые столбцы (например, «street», «homeFacts», «schools», «zipcode», «state») практически не содержат пропусков – здесь можно ожидать надёжное качество данных.

В целом, анализ показывает, что для оптимальной работы модели потребуется уделить повышенное внимание обработке столбцов с большим количеством отсутствующих значений.


In [6]:
# Объединяем столбцы "private pool" и "PrivatePool" с помощью combine_first.
data['pool_encoded'] = data['private pool'].combine_first(data['PrivatePool'])
# Предполагаем, что отсутствие информации означает отсутствие бассейна, - заменяем NaN на False.
data['pool_encoded'] = data['pool_encoded'].fillna(False)
# Приводим текстовые обозначения наличия бассейна к булевому типу: 'yes' и 'Yes' -> True
data['pool_encoded'] = data['pool_encoded'].replace(['yes', 'Yes'], True)
# Удаляем исходные столбцы, так как вся необходимая информация теперь в pool_encoded.
data.drop(['private pool', 'PrivatePool', 'mls-id', 'MlsId'], axis=1, inplace=True)
# Выводим уникальные значения объединённого столбца и первые строки для проверки корректности обработки.
print("Уникальные значения 'pool_encoded':", data['pool_encoded'].unique())
display(data.head())

Уникальные значения 'pool_encoded': [False  True]


C:\Users\Koop_\AppData\Local\Temp\ipykernel_16472\2208532096.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['pool_encoded'] = data['pool_encoded'].replace(['yes', 'Yes'], True)


,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool_encoded
0,Active,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,"$418,000",False
1,for sale,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,"$310,000",False
2,for sale,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,"$2,895,000",True
3,for sale,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,"$2,395,000",False
4,for sale,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,"$5,000",False


In [7]:
# Функция для удаления дубликатов из DataFrame с выводом статистики
def remove_duplicates(data):
    """
    Удаляет дубликаты из DataFrame и выводит количество удалённых строк.
    
    :param data: DataFrame с исходными данными.
    :return: DataFrame без дубликатов.
    """
    rows_before = data.shape[0]
    data_clean = data.drop_duplicates(ignore_index=True)
    rows_after = data_clean.shape[0]
    duplicates_removed = rows_before - rows_after
    
    print(f"Удалено {duplicates_removed} дубликатов (из {rows_before} строк -> {rows_after} строк)")
    return data_clean

# Применяем функцию
data = remove_duplicates(data)

Удалено 91 дубликатов (из 377135 строк -> 377044 строк)


## Работа над признаком status

In [8]:
# Подсчёт пропущенных значений в столбце 'status'
missing_values_count = data['status'].isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100

# Получаем информацию по уникальным значениям
unique_status_count = data['status'].nunique()
unique_status_values = data['status'].unique()[:10]

# Выводим статистику столбца 'status' в удобном виде
print("Статистика столбца 'status':")
print(f"  Количество пропущенных значений: {missing_values_count}")
print(f"  Процент пропущенных значений: {missing_values_percentage:.2f}%")
print(f"  Количество уникальных значений: {unique_status_count}")
print(f"  Первые 10 уникальных значений: {list(unique_status_values)}")

Статистика столбца 'status':
  Количество пропущенных значений: 39916
  Процент пропущенных значений: 10.59%
  Количество уникальных значений: 159
  Первые 10 уникальных значений: ['Active', 'for sale', nan, 'New construction', 'New', 'For sale', 'Pending', 'P', 'Active/Contingent', 'Pre-foreclosure / auction']


Разделим статусы на категории а пропуски заменим на other и запишем их

In [9]:
# Задаём словарь категорий для статусов
categories = {
    "For Sale": ["for sale", "For sale", "New construction", "New"],
    "Active": [
        "Active", "A Active", "Active/Contingent", "Active Under Contract", "Active Option", "Auction - Active",
        "Active With Contingencies", "Active Option Contract", "Active Contingency", "Active Backup",
        "Active Contingent", "Active - Auction", "Active With Offer", "Active - Contingent", "Active with Contract",
        "Temporary Active", "Re Activated", "Reactivated"
    ],
    "Pending": [
        "P", "Pending", "pending", "P Pending Sale", "Pending Ab", "Pending Continue To Show",
        "Pending Inspection", "Pending Offer Approval", "Pending In", "Pending W/Insp Finance", "Pending Fe",
        "Pending W/Backup Wanted", "Pending Backups Wanted", "Pending With Contingencies", "Lease/Purchase Pending",
        "Pending Bring Backup", "Pending - Taking Backups", "Pending - Continue to Show",
        "Pending Taking Backups", "Offer Pending Signature", "Pending (Do Not Show)", "Pending W/ Cont.",
        "Pending W/Escape Clause", "Pending - Backup Offer Requested", "Pending Sale"
    ],
    "Contingent": [
        "Contingent", "Contingent Finance And Inspection", "Contingent Show",
        "Contingent Take Backup", "Contingent - Sale of Home", "Contingent Finance and Inspection",
        "C Continue Show", "Contingent   Show", "Contingent   Release", "Contingent   No Show",
        "CT Insp - Inspection Contingency", "Contingent   Foreclosure", "Conting Accpt Backups",
        "Contingent - Financing", "Contingency 48 Hr (+/ )", "Contingency Contract", "Contingent Escape"
    ],
    "Foreclosure": [
        "Pre-foreclosure", "Pre-foreclosure / auction", " / auction", "Foreclosed", "foreclosure", "Foreclosure"
    ],
    "Under Contract": [
        "Under Contract", "Under Contract   Showing", "Under Contract Backups", "Under Contract Show",
        "Under Contract - Show", "Under Contract - No Show", "Under contract", "U Under Contract",
        "Due Diligence Period", "Contract Contingent On Buyer Sale", "Contract P", "Ct", "Uc Continue To Show",
        "Under Contract Taking Back Up Offers", "Under Contract W/ Bckp", "Contract"
    ],
    "For Rent": ["for rent", "Apartment for rent", "Condo for rent"],
    "Auction": ["Auction"],
    "Recently Sold": ["recently sold"],
    "Price Change": ["Price Change"],
    "Back on Market": ["Back on Market", "Back On Market"],
    "Closed": ["Closed"],
    "Listing Extended": ["Listing Extended"],
    "Coming Soon": [
        "Coming soon: Nov 21.", "Coming soon: Dec 4.", "Coming soon: Nov 23.", "Coming soon: Nov 29.",
        "Coming soon: Dec 2.", "Coming soon: Dec 10.", "Coming soon: Dec 24.", "Coming soon: Nov 14.",
        "Coming soon: Nov 22.", "Coming soon: Oct 21.", "Coming soon: Dec 14.", "Coming soon: Oct 24.",
        "Coming soon: Dec 18.", "Coming soon: Dec 16.", "Coming soon: Dec 3.", "Coming soon: Dec 25.",
        "Coming soon: Nov 11.", "Coming soon: Nov 28.", "Coming soon: Nov 17.", "Coming soon: Dec 6.",
        "Coming soon: Nov 27.", "Coming soon: Nov 26.", "Coming soon: Dec 7.", "Coming soon: Dec 27.",
        "Coming soon: Dec 11.", "Coming soon: Dec 5.", "Coming soon: Nov 13.", "Coming soon: Nov 19.",
        "Coming soon: Nov 8.", "Coming soon: Oct 29.", "Coming soon: Dec 15.", "Coming soon: Oct 30.",
        "Coming soon: Dec 9.", "Coming soon: Dec 20.", "Coming soon: Dec 13.", "Coming soon: Dec 23.",
        "Coming soon: Nov 30.", "Coming soon: Dec 1.", "Coming soon: Nov 5.", "Coming soon: Nov 12.",
        "Coming soon: Nov 25.", "Coming soon: Nov 9."
    ],
}


def categorize_status(status_value):
    """
    Функция для классификации статуса объекта недвижимости по заданным категориям.
    Если статус не найден в словаре, возвращает "Other".
    """
    for category, values in categories.items():
        if status_value in values:
            return category
        
    return "Other"

# Применяем категоризацию к столбцу 'status'
data['status'] = data['status'].apply(categorize_status)

# Получаем статистику по столбцу 'status'
missing_values_count = data['status'].isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100
unique_status = list(data['status'].unique())
unique_status_count = data['status'].nunique()

print("Статистика столбца 'status':")
print(f"  Количество пропущенных значений: {missing_values_count}")
print(f"  Процент пропущенных значений: {missing_values_percentage:.2f}%")
print(f"  Количество уникальных значений: {unique_status_count}")
print(f"  Первые 10 уникальных значений: {unique_status[:10]}")

# Сохраняем количество строк до удаления дубликатов
rows_before = data.shape[0]
# Удаляем дубликаты с переиндексацией
data = data.drop_duplicates(ignore_index=True)
# Сохраняем количество строк после удаления дубликатов и рассчитываем разницу
rows_after = data.shape[0]
duplicates_removed = rows_before - rows_after
print(f'\nУдалено {duplicates_removed} дубликатов')

Статистика столбца 'status':
  Количество пропущенных значений: 0
  Процент пропущенных значений: 0.00%
  Количество уникальных значений: 15
  Первые 10 уникальных значений: ['Active', 'For Sale', 'Other', 'Pending', 'Foreclosure', 'Under Contract', 'For Rent', 'Auction', 'Contingent', 'Price Change']

Удалено 0 дубликатов


## Работа над признаком propertyType

Анализ исходного признака propertyType

In [10]:
# Анализ признака propertyType
missing_values_count = data.propertyType.isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100
print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {data.propertyType.nunique()}\n"
      f"Уникальные значения: {list(data.propertyType.unique()[:10])}")

Количество пропущенных значений: 34725
Процент пропущенных значений: 9.21%
Количество уникальных значений: 1280
Уникальные значения: ['Single Family Home', 'single-family home', 'lot/land', 'townhouse', 'Florida', nan, 'Single Family', 'coop', 'English', '2 Story']


In [11]:
# Приведение признака propertyType к нижнему регистру
data.propertyType = data.propertyType.str.lower()

# Приводим к единому виду написания и заменяем 'yes' и 'unknown' на 'other'
data.propertyType = data.propertyType \
    .str.replace('single-family home', 'single family', regex=False) \
    .str.replace('single family home', 'single family', regex=False) \
    .str.replace('yes', 'other', regex=False) \
    .str.replace('unknown', 'other', regex=False)

# Извлекаем первую часть значения, если в строке несколько типов, разделенных запятыми
data['Type'] = data['propertyType'].str.split(',').str[0]

In [12]:
# Проверяем полученные значения в столбце 'Type'
missing_values_count = data.Type.isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100
print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {data.Type.nunique()}\n"
      f"Уникальные значения: {list(data.Type.unique()[:10])}")

Количество пропущенных значений: 34725
Процент пропущенных значений: 9.21%
Количество уникальных значений: 255
Уникальные значения: ['single family', 'lot/land', 'townhouse', 'florida', nan, 'coop', 'english', '2 story', 'multi-family', 'penthouse']


In [13]:
# Определяем словарь для замены синонимов типов недвижимости
synonyms_mapping = {
    'single_family_home': [
        'single family', '1 story', '2 story', 'detached', '1 story/ranch', '1 story traditional', 'single detached',
        'singlefamilyresidence', 'single wide', 'single-wide mobile with land', 'two story', 'one story',
        'one level unit', 'rancher', '1 1/2 story', 'single wide mh', 'one story traditional'
    ],
    'multi_family_home': [
        'multi-family', 'multi-family home', 'duplex', 'triplex', 'fourplex', 'multi_level', 'multi family',
        '2 unit condo', '2-story'
    ],
    'condo': [
        'condo', 'coop', 'cooperative', 'condo/townhome/row home/co-op', 'condo/townhome', 'condominium', 'condo/unit',
        'apartment/condo/townhouse', 'co-op', '2 story condo', 'high rise'
    ],
    'townhouse': [
        'townhouse', 'townhome style', 'townhouse-interior', 'townhouse-end unit'
    ],
    'apartment': [
        'apartment', 'condominium (single level)', 'high-rise', 'mid-rise', 'low-rise (1-3 stories)', 'flats', 'studio'
    ],
    'land': [
        'lot/land', 'land'
    ],
    'mobile_home': [
        'mobile/manufactured', 'mobile / manufactured', 'manufactured house', 'mfd/mobile home', 'mobile home',
        'manufactured home', 'manufactured double-wide', 'manufactured single-wide', 'mobile home 1 story',
        'mobile manu - double wide'
    ],
    'miscellaneous': [
        'miscellaneous'
    ],
    'ranch': [
        'ranch', 'one story'
    ],
    'modern': [
        'contemporary', 'contemporary/modern', 'modern', 'mid-century modern', 'modern farmhouse', 'modernist'
    ],
    'historical': [
        'historical', 'designated historical home', 'historical/conservation district', 'historic/older',
        'historic vintage', 'historic'
    ],
    'other': [
        'other', 'english', 'urban contemporary', 'other style', 'florida', 'farms/ranches', 'carriage house',
        'country english', 'straight thru', 'less than 4 floors', 'bungalow', 'custom', 'arts & crafts', 'tudor',
        'new build 2019', 'split foyer', 'cottage', 'cottage/camp', 'garden home', 'farm/ranch', 'farm/ranch house',
        'farm house', 'hi ranch', 'attached duplex', 'farmhouse', 'houseboat', 'ground floor', 'victorian',
        '3 story', '3+ stories', 'santa barbara/tuscan', 'old style', 'modular/prefab', 'post and beam',
        'manuf/mobile', 'multiple occupancy', 'attached', 'hawaiian plantation', 'forest garden home',
        '1 1/2 story with basement', 'split-entry', 'texas hill country', 'lake house', '1 story with basement',
        'hi-rise', 'coastal beach home', 'historical', 'key west/coastal', 'loft/balcony', 'english manor',
        'mid-rise (4-7 stories)', 'mid-level', 'new englander', 'residential (<1 acre)', 'ranch',
        'residential (1+ acre)', 'split', 'split level', 'split (4 level)', 'split (5+ level)', 'urban', 'patio',
        'patio home', 'penthouse', 'manor', 'victorian/federal', 'coastal', 'coastal contemporary', 'coastal ii',
        'coastal modern', 'coastal two story', 'mountain contemporary', 'key west', 'high ranch', 'end unit'
    ]
}

Применение синонимической замены

In [14]:
# Функция для замены синонимов
def replace_synonyms(value):
    if pd.isna(value):
        return "other"
    value = value.strip()  # удаляем лишние пробелы
    for key, synonym_values in synonyms_mapping.items():
        if value in synonym_values:
            return key
    return "other"
# Применяем функцию к столбцу Type
data['Type'] = data['Type'].apply(replace_synonyms)

Финальная проверка и удаление исходного признака propertyType

In [15]:
# Проверка столбца 'Type' после замены синонимов
missing_values_count = data.Type.isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100
print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {data.Type.nunique()}\n"
      f"Уникальные значения: {list(data.Type.unique()[:10])}")
# После успешного преобразования исходный столбец propertyType больше не нужен
data.drop('propertyType', axis=1, inplace=True)

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 12
Уникальные значения: ['single_family_home', 'land', 'townhouse', 'other', 'condo', 'multi_family_home', 'mobile_home', 'apartment', 'modern', 'miscellaneous']


## Работа над признаком street

In [16]:
# Проверка пропусков и уникальных значений в столбце 'street'
missing_values_count = data['street'].isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100
print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {data['street'].nunique()}")
# Удаляем строки, где отсутствует информация об адресе
data = data.dropna(subset=['street'])

Количество пропущенных значений: 2
Процент пропущенных значений: 0.00%
Количество уникальных значений: 337076


Признаки, в которых все значения уникальны, могут быть бесполезны для машинного обучения, так как такие признаки имеют нулевую дисперсию.


In [17]:
# Удаляем столбец 'street'
data = data.drop('street', axis=1)

## Работаем с признаком baths

Предварительный анализ столбца baths

In [18]:
# выполним проверку
missing_values_count = data.baths.isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {data.baths.nunique()}\n"
      f"Уникальные значения: {list(data.baths.unique()[:10])}")

Количество пропущенных значений: 106260
Процент пропущенных значений: 28.18%
Количество уникальных значений: 229
Уникальные значения: ['3.5', '3 Baths', '2 Baths', '8 Baths', nan, '2', '3', 'Bathrooms: 2', '1,750', '4 Baths']


In [19]:
# Удаляем все буквы, пробелы и двоеточия
data.baths = data.baths.str.replace('[a-zA-Z:]','', regex=True)
# Удаляем все пробелы
data.baths = data.baths.str.replace(' ','', regex=True)

In [20]:
# выполним проверку
missing_values_count = data.baths.isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {data.baths.nunique()}\n"
      f"Уникальные значения: {list(data.baths.unique()[:10])}")

Количество пропущенных значений: 106260
Процент пропущенных значений: 28.18%
Количество уникальных значений: 148
Уникальные значения: ['3.5', '3', '2', '8', nan, '1,750', '4', '5', '1,000', '7']


пропуски заполним нулями так как большинство пропущенных значений соответствуют земле где нет здания

In [21]:
# поправим кривые значения
data['baths'] = data['baths'].str.replace('[a-zA-Z+:]','', regex=True)
data['baths'].replace(['nan','', '~', '..', '--', '—','0.0','0.00','0/0'], 0, inplace=True)
data['baths'].replace(['1-2','1/1-0/1-0/1-0', '1/1/1/1','1-0/1-0/1', ], 1, inplace=True)
data['baths'].replace(['2-1/2-1/1-1/1-1','3-1/2-2',], 2, inplace=True)
data['baths'].replace(['116/116/116'], 116, inplace=True)
data['baths'].replace(['7,500'], 7.5, inplace=True)
data['baths'].replace(['5,000'], 5, inplace=True)
data['baths'].replace(['3,500'], 3.5, inplace=True)
data['baths'].replace(['2,750'], 2.75, inplace=True)
data['baths'].replace(['2,250'], 2.25, inplace=True)
data['baths'].replace(['1,250'], 1.25, inplace=True)
data['baths'].replace(['1,500'], 1.5, inplace=True)
data['baths'].replace(['4,000'], 4, inplace=True)
data['baths'].replace(['2,000'], 2, inplace=True)
data['baths'].replace(['3,000'], 3, inplace=True)
data['baths'].replace(['2,500'], 2.5, inplace=True)
data['baths'].replace(['1,000'], 1, inplace=True)
data['baths'].replace(['1,750'], 1.75, inplace=True)
data['baths'] = data['baths'].fillna(0)

C:\Users\Koop_\AppData\Local\Temp\ipykernel_16472\979716405.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['baths'].replace(['nan','', '~', '..', '--', '—','0.0','0.00','0/0'], 0, inplace=True)


In [22]:
# переведем в числовой признак и округлим до целого числа
data['baths'] = data['baths'].astype(float).round()

In [23]:
# Отбираем строки по заданным условиям при помощи логических операций (& - "и", | - "или", ~ - "не")
selected_rows = data[(data['baths'] == 0) & ((data['Type'] == 'other') | (data['Type'] == 'land') | (data['Type'] == 'ranch'))]

# Выводим полученные строки
display(selected_rows.head())
# Находим процент выбранных строк от общего числа строк в исходном датафрейме
percent_selected = len(selected_rows) / len(data) * 100
# Выводим полученные строки и процент выбранных строк
print(f"Процент выбранных строк от общего числа строк: {percent_selected:.2f}%")

,status,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool_encoded,Type
4,For Sale,0.0,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,"$5,000",False,land
6,Active,0.0,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",NaN,POINCIANA,"[{'rating': ['3', '3', '1', 'NR'], 'data': {'D...","1,507",34759,NaN,FL,One,"181,500",False,other
7,Active,0.0,"{'atAGlanceFacts': [{'factValue': '1976', 'fac...",NaN,Memphis,"[{'rating': ['4', '2', '2'], 'data': {'Distanc...",NaN,38115,NaN,TN,NaN,"68,000",False,other
11,Active,0.0,"{'atAGlanceFacts': [{'factValue': '2015', 'fac...",NaN,Houston,"[{'rating': ['6', '3', '6', '1', '5'], 'data':...","3,130",77068,NaN,TX,2,"260,000",False,other
40,Active,0.0,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,ROTONDA WEST,"[{'rating': ['6', '6', '7'], 'data': {'Distanc...",NaN,33947,NaN,FL,NaN,"54,000",False,other


Процент выбранных строк от общего числа строк: 19.36%


In [24]:
selected_rows = data[(data['baths'] == 0) & ~(data['Type'].isin(['other', 'land', 'ranch']))]
display(selected_rows.head())
# Находим уникальные значения признака status в выбранных строках
unique_statuses = selected_rows['Type'].unique()
# Выводим полученные строки и процент выбранных строк
print(f"Уникальные значения признака Type в выбранных строках: {unique_statuses}")
percent_selected = len(selected_rows) / len(data) * 100
print(f"Процент выбранных строк от общего числа строк: {percent_selected:.2f}%")

,status,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool_encoded,Type
5,For Sale,0.0,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Philadelphia,"[{'rating': [], 'data': {'Distance': [], 'Grad...",897 sqft,19145,2 Beds,PA,2.0,"$209,000",False,townhouse
15,For Sale,0.0,"{'atAGlanceFacts': [{'factValue': '1905', 'fac...",NaN,Brooklyn,"[{'rating': ['8', '8', '3'], 'data': {'Distanc...",NaN,11219,NaN,NY,NaN,"1,650,000",False,single_family_home
17,Active,0.0,"{'atAGlanceFacts': [{'factValue': '1899', 'fac...",NaN,New York,"[{'rating': ['2', '5', '1', '2'], 'data': {'Di...","3,325",10027,NaN,NY,NaN,"2,650,000",False,townhouse
35,For Sale,0.0,"{'atAGlanceFacts': [{'factValue': '1981', 'fac...",NaN,Charlotte,"[{'rating': ['7/10', '4/10', '8/10'], 'data': ...",900 sqft,28202,2 Beds,NC,NaN,"$260,000",False,condo
71,For Sale,0.0,"{'atAGlanceFacts': [{'factValue': '1918', 'fac...",NaN,Seattle,"[{'rating': ['6/10', '3/10', '4/10'], 'data': ...",770 sqft,98106,2 Beds,WA,1.0,"$475,000",False,single_family_home


Уникальные значения признака Type в выбранных строках: ['townhouse' 'single_family_home' 'condo' 'apartment' 'modern'
 'miscellaneous' 'multi_family_home' 'mobile_home' 'historical']
Процент выбранных строк от общего числа строк: 10.25%


In [25]:
# выполним проверку
missing_values_count = data.baths.isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {data.baths.nunique()}\n"
      f"Уникальные значения: {list(data.baths.unique()[:10])}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 49
Уникальные значения: [4.0, 3.0, 2.0, 8.0, 0.0, 5.0, 1.0, 7.0, 6.0, 9.0]


Итого имеем 29.61% мы заменили значением 0 из них 19.45% это ранчо чемля и иной тип что в полне может быть правдой и 10.10% 'таунхаус' 'дом на одну семью' 'кондоминиум' 'квартира' 'современный' 'разное' 'многосемейный_дом' 'мобильный_дом' 'исторический' из них минимум половина действительно может не иметь ванной комнаты, в целом такой результат можно считать приемлемым

## Работаем над признаком homeFacts

In [26]:
# выполним проверку
missing_values_count = data.homeFacts.isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {data.homeFacts.nunique()}\n"
f"Уникальные значения: {list(data.homeFacts.unique()[:1])}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 321008
Уникальные значения: ["{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': '$144', 'factLabel': 'Price/sqft'}]}"]


In [27]:
label = data.homeFacts.str.findall(r"\bfactLabel': ([\s\S]+?)[}\b]")
label[100]

["'Year built'",
 "'Remodeled year'",
 "'Heating'",
 "'Cooling'",
 "'Parking'",
 "'lotsize'",
 "'Price/sqft'"]

In [28]:
Value = data.homeFacts.str.findall(r"\bfactValue': ([\s\S]+?), 'factLabel\b")
Value[:10]

0    ['2019', '', 'Central A/C, Heat Pump', '', '',...
1    ['2019', '', '', '', '', '5828 sqft', '$159/sq...
2    ['1961', '1967', 'Forced Air', 'Central', 'Att...
3    ['2006', '2006', 'Forced Air', 'Central', 'Det...
4            ['', '', '', '', '', '10,019 sqft', None]
5    ['1920', '', 'Forced Air', 'Central', '', '680...
6    ['2006', '2006', 'Electric, Heat Pump', 'Centr...
7    ['1976', '', '', '', '', '8,750 Sq. Ft.', '$57...
8    ['1970', '', 'Forced Air', 'Central', '', '124...
9    ['2019', None, 'Gas', 'Central', 'Attached Gar...
Name: homeFacts, dtype: object

In [29]:
# создадим список названий признаков, удалив лишние кавычки
list_label = ','.join(label[0]).replace("'","").split(',')
list_label

['Year built',
 'Remodeled year',
 'Heating',
 'Cooling',
 'Parking',
 'lotsize',
 'Price/sqft']

In [30]:
# Создадим эти признаки в нашем фрейме и заполним значениями
for i, val in enumerate(list_label):
    data[val]=Value.apply(lambda x: x[i])

In [31]:
# Теперь признак homeFacts можно удалить
data = data.drop('homeFacts', axis=1)

В результате выполнения этого кода данные из столбца homeFacts будут разбиты на отдельные столбцы по атрибутам, таким как 'Year built', 'Remodeled year', 'Heating' и т.д., что позволит использовать их для дальнейшего анализа и моделирования.

## Разбираем признак Year built 

In [32]:
# Выведем первоначальные уникальные значения столбца "Year built" для ознакомления
print("Первичные уникальные значения 'Year built' (первые 10):")
print(sorted(data['Year built'].unique())[:10])

Первичные уникальные значения 'Year built' (первые 10):
["''", "'1'", "'1019'", "'1057'", "'1060'", "'1208'", "'1700'", "'1703'", "'1735'", "'1740'"]


In [33]:
# Удаляем лишние кавычки
data['Year built'] = data['Year built'].str.replace("'", "", regex=True)
# Замена пустых строк на 'no date'
data['Year built'] = data['Year built'].str.replace(r"^\s*$", "no date", regex=True)
# Определяем список некорректных значений и заменяем их на 'no date'
invalid_values = ['1', '1208', '559990649990', '2025', 'No Data', 'None']
data.loc[data['Year built'].isin(invalid_values), 'Year built'] = 'no date'
# Исправляем опечатки
corrections = {'1019': '1919', '1057': '1957', '1060': '1960'}
data['Year built'] = data['Year built'].replace(corrections)
# Выводим результат обработки
print("Обработка столбца 'Year built' завершена.")
print("Уникальные значения 'Year built' (первые 10):")
print(sorted(data['Year built'].unique())[:10])

Обработка столбца 'Year built' завершена.
Уникальные значения 'Year built' (первые 10):
['1700', '1703', '1735', '1740', '1750', '1780', '1788', '1790', '1794', '1795']


In [34]:
# Отбираем строки, где "Year built" равен 'no date' и тип недвижимости (Type) равен 'other', 'land' или 'ranch'
selected_rows = data[(data['Year built'] == 'no date') & (data['Type'].isin(['other', 'land', 'ranch']))]
# Выводим первые несколько строк отобранных данных
print("Отобранные строки:")
display(selected_rows.head())
# Рассчитываем процент выбранных строк от общего числа строк
percent_selected = (len(selected_rows) / len(data)) * 100
print(f"\nВыбрано строк: {len(selected_rows)}")
print(f"Процент выбранных строк от общего числа: {percent_selected:.2f}%")

Отобранные строки:


,status,baths,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool_encoded,Type,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft
4,For Sale,0.0,NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,"$5,000",False,land,no date,'','','','',"'10,019 sqft'",None
26,Active,1.0,NaN,Miami,"[{'rating': ['9', '10', 'NR', '6', '3'], 'data...",NaN,33183,Baths,FL,NaN,"799,000",False,other,no date,'','','','','—',None
40,Active,0.0,NaN,ROTONDA WEST,"[{'rating': ['6', '6', '7'], 'data': {'Distanc...",NaN,33947,NaN,FL,NaN,"54,000",False,other,no date,'','','','','—',None
41,For Sale,0.0,NaN,San Diego,"[{'rating': ['9/10', '9/10', '9/10'], 'data': ...",NaN,92127,NaN,CA,NaN,"$1,100,000",False,land,no date,'','','','','9.25 acres',None
42,Active,0.0,NaN,New York,"[{'rating': ['NR', 'NR', '9', '8', '8', '2', '...",NaN,10006,NaN,NY,NaN,"850,000",False,other,no date,'2003','','','','—',None



Выбрано строк: 44783
Процент выбранных строк от общего числа: 11.88%


Таким образом, в результате обработки столбца «Year built» все некорректные и пропущенные значения заменены на 'no date', а опечатки скорректированы.

In [35]:
# Фильтруем строки: 'Year built' равен 'no date' и 'Type' не входит в ['other', 'land', 'ranch']
selected_rows = data[(data['Year built'] == 'no date') & (~data['Type'].isin(['other', 'land', 'ranch']))]
display(selected_rows.head())

,status,baths,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool_encoded,Type,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft
27,For Sale,3.0,NaN,Houston,"[{'rating': ['6/10', '9/10', '3/10'], 'data': ...","2,575 sqft",77018,3 Beds,TX,NaN,"$499,900",False,single_family_home,no date,'','','','','3828 sqft','$194/sqft'
57,For Sale,2.0,NaN,Durham,"[{'rating': ['3/10', '4/10', '3/10'], 'data': ...","1,681 sqft",27713,3 Beds,NC,2.0,"$259,658",False,townhouse,no date,'','','','','','$154/sqft'
59,For Sale,2.0,NaN,Temple Terrace,"[{'rating': ['2/10', '3/10', '2/10'], 'data': ...","1,504 sqft",33637,3 Beds,FL,1.0,"$244,990",False,single_family_home,no date,'','','','','','$163/sqft'
83,For Sale,3.0,NaN,Spring,"[{'rating': ['9/10', '9/10', '8/10'], 'data': ...","2,265 sqft",77386,4 Beds,TX,1.0,"$299,999",False,single_family_home,no date,'','','','','','$132/sqft'
91,Active,2.0,NaN,Port Charlotte,"[{'rating': ['5', '4', '6', '1', 'NR', 'NR', '...","1,702",33953,3,FL,NaN,"$235,900",False,single_family_home,no date,'','','','',None,'$139'


In [36]:
# Находим уникальные значения признака 'Type' в выбранных строках
unique_types = selected_rows['Type'].unique()
# Вычисляем процент выбранных строк от общего числа строк
percent_selected = len(selected_rows) / len(data) * 100
# Вывод результатов
print("Уникальные значения признака 'Type' в выбранных строках:", unique_types)
print("Количество выбранных строк:", len(selected_rows))
print(f"Процент выбранных строк от общего числа строк: {percent_selected:.2f}%")

Уникальные значения признака 'Type' в выбранных строках: ['single_family_home' 'townhouse' 'condo' 'miscellaneous'
 'multi_family_home' 'apartment' 'mobile_home' 'modern']
Количество выбранных строк: 18700
Процент выбранных строк от общего числа строк: 4.96%


In [37]:
# выполним проверку
missing_values_count = data['Year built'].isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {data['Year built'].nunique()}\n"
f"Уникальные значения: {list(data['Year built'].sort_values().unique()[:10])}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 222
Уникальные значения: ['1700', '1703', '1735', '1740', '1750', '1780', '1788', '1790', '1794', '1795']


In [38]:
# посмотрим на колличество no date в %
print(f"Процент no date: {round(data[data['Year built']=='no date'].shape[0]/data.shape[0]*100, 2)}%")
print(f"Количество no date: {data.loc[data['Year built']=='no date', 'Year built'].count()}")

Процент no date: 16.84%
Количество no date: 63483


Большая часть значений "no date" соответствует типу "земля ранчо" (11.88%), что выглядит вполне правдоподобно. Для остальных типов "no date" составляют 4.96% – это достаточно много, чтобы негативно сказаться на качестве данных, но при этом удалять такие строки было бы слишком рискованно, а заполнение года постройки случайными значениями не оправдано. Поэтому лучше оставить эти значения как есть.

## Разбираем признак Remodeled year

In [39]:
# Удаляем лишние одинарные кавычки
data['Remodeled year'] = data['Remodeled year'].str.replace("'", "", regex=True)
# Просмотр уникальных значений до замены некорректных записей
unique_before = data['Remodeled year'].sort_values().unique()
print("Уникальные значения до обработки:", unique_before[:10])

Уникальные значения до обработки: ['' '0' '1111' '1738' '1800' '1845' '1846' '1853' '1862' '1869']


In [40]:
# Определяем список некорректных значений
incorrect_vals = ['', '0', '1111']
# Заменяем указанные значения на 'None'
data.loc[data['Remodeled year'].isin(incorrect_vals), 'Remodeled year'] = 'None'
# Просмотр уникальных значений после замены
unique_after = data['Remodeled year'].sort_values().unique()
print("Уникальные значения после замены:", unique_after[:10])

Уникальные значения после замены: ['1738' '1800' '1845' '1846' '1853' '1862' '1869' '1870' '1874' '1876']


In [41]:
# Проверим количество пропущенных (NaN) значений
missing_count = data['Remodeled year'].isna().sum()
total_rows = len(data)
missing_percentage = (missing_count / total_rows) * 100
print(f"Количество пропущенных значений: {missing_count}")
print(f"Процент пропущенных значений: {missing_percentage:.2f}%")
print(f"Количество уникальных значений: {data['Remodeled year'].nunique()}")
print(f"Уникальные значения: {list(unique_after[:10])}")
# Подсчитаем, сколько записей равно строке 'None'
none_count = data[data['Remodeled year'] == 'None'].shape[0]
none_percentage = round((none_count / total_rows) * 100, 2)
print(f"Процент 'None': {none_percentage}%")
print(f"Количество 'None': {none_count}")

Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 152
Уникальные значения: ['1738', '1800', '1845', '1846', '1853', '1862', '1869', '1870', '1874', '1876']
Процент 'None': 60.0%
Количество 'None': 226240


In [42]:
# Если полученный процент 'None' слишком велик (например, 60% как в примере), удаляем признак
data = data.drop('Remodeled year', axis=1)
print("Признак 'Remodeled year' удалён. Текущие столбцы:", list(data.columns))

Признак 'Remodeled year' удалён. Текущие столбцы: ['status', 'baths', 'fireplace', 'city', 'schools', 'sqft', 'zipcode', 'beds', 'state', 'stories', 'target', 'pool_encoded', 'Type', 'Year built', 'Heating', 'Cooling', 'Parking', 'lotsize', 'Price/sqft']


## Обрабатываем признак heating

In [43]:
# Удаляем лишние кавычки и приводим строки к нижнему регистру
data['Heating'] = data['Heating'].str.replace("'", "", regex=True).str.lower()
# Выводим топ-10 наиболее частых значений
print("Распределение значений Heating (до кодирования):")
print(data['Heating'].value_counts().head(10))
# Кодируем значение: если отопление указано (не пусто, не "no data", не "none"), то True, иначе False
data['Heating_encoded'] = data['Heating'].apply(lambda x: True if x not in ['', 'no data', 'none'] else False)
# Проверим результаты кодирования
print("\nПроверка показателей Heating_encoded:")
missing_values_count = data['Heating_encoded'].isna().sum()
total_count = len(data)
missing_percentage = (missing_values_count / total_count) * 100
unique_values = data['Heating_encoded'].unique()
print(f"Количество пропущенных значений: {missing_values_count}")
print(f"Процент пропущенных значений: {missing_percentage:.2f}%")
print(f"Количество уникальных значений: {len(unique_values)}")
print(f"Уникальные значения: {sorted(unique_values)}")
# Удаляем исходный признак Heating, так как он заменён новыми данными
data = data.drop('Heating', axis=1)
print("\nСтолбец 'Heating' удалён.")

Распределение значений Heating (до кодирования):
Heating
forced air          134307
                    105670
other                29622
electric             10216
gas                   9295
heat pump             8851
no data               8610
central air           7814
central electric      7112
central               6247
Name: count, dtype: int64

Проверка показателей Heating_encoded:
Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 2
Уникальные значения: [False, True]

Столбец 'Heating' удалён.


После обработки признак Heating был очищен от лишних символов, сконвертирован в нижний регистр, и создан булев признак Heating_encoded. Записи с пустыми значениями или помеченные как "no data"/"none" получили значение False, а все остальные – True. Исходный столбец удалён.

## обработаем признак cooling

In [44]:
# Удаляем лишние кавычки и приводим значения к нижнему регистру
data['Cooling'] = data['Cooling'].str.replace("'", "", regex=True).str.lower()
# Выводим распределение значений Cooling (топ-10)
print("Распределение значений Cooling (до кодирования):")
print(data['Cooling'].value_counts().head(10))
# Кодируем признак Cooling аналогично Heating:
# Если присутствует значение, отличное от ['', 'no data', 'none'], то считаем, что охлаждение имеется (True)
data['Cooling_encoded'] = data['Cooling'].apply(lambda x: True if x not in ['', 'no data', 'none'] else False)
# Проверяем результаты кодирования - можно вывести уникальные значения
print("\nПроверка показателей Cooling_encoded:")
print(data['Cooling_encoded'].value_counts())
# Удаляем исходный признак Cooling
data = data.drop('Cooling', axis=1)
print("\nСтолбец 'Cooling' удалён.")

Распределение значений Cooling (до кодирования):
Cooling
central             158742
                    120306
central air          14384
none                 10923
no data              10615
has cooling           9730
central electric      6154
wall                  4017
central gas           3573
central heating       2807
Name: count, dtype: int64

Проверка показателей Cooling_encoded:
Cooling_encoded
True     235198
False    141844
Name: count, dtype: int64

Столбец 'Cooling' удалён.


Признак Cooling обработан аналогично Heating – значения очищены и переведены в нижний регистр, после чего создан новый булев признак Cooling_encoded, который принимает True, если охлаждение присутствует, и False в противном случае. Исходный столбец Cooling удаляется, чтобы избежать дублирования информации.

## Обработаем признак Parking

In [45]:
# Приводим значения столбца 'Parking' к нижнему регистру и удаляем лишние кавычки
data['Parking'] = data['Parking'].str.replace("'", "", regex=True).str.lower()
# Выводим распределение значений (топ-10)
print("Распределение значений 'Parking' до кодирования:")
print(data['Parking'].value_counts().head(10))
# Кодируем признак: считаем, что наличие парковки (True), если значение не равно пустой строке,
# не 'no data', не 'none' и не '0'. В противном случае считаем как отсутствие парковки (False)
data['Parking_encoded'] = data['Parking'].apply(lambda x: True if x not in ['', 'no data', 'none', '0'] else False)
# Выводим распределение нового признака
print("\nРаспределение значений 'Parking_encoded':")
print(data['Parking_encoded'].value_counts())
# Удаляем исходный столбец 'Parking'
data = data.drop('Parking', axis=1)
print("\nСтолбец 'Parking' удалён.")

Распределение значений 'Parking' до кодирования:
Parking
                   171755
attached garage     70748
2 spaces            28061
1 space             14252
no data             13332
detached garage     13200
carport              7743
none                 5901
off street           5279
3 spaces             4724
Name: count, dtype: int64

Распределение значений 'Parking_encoded':
Parking_encoded
False    192102
True     184940
Name: count, dtype: int64

Столбец 'Parking' удалён.


В результате обработки признак Parking был очищен от лишних кавычек и приведен к нижнему регистру. Создан новый бинaрный признак Parking_encoded, где значения True означают наличие парковки, а False – отсутствие информации о парковке (предполагается, что пропуски означают отсутствие). Исходный столбец удален.

## Обработаем признак lotsize

In [46]:
# Выводим распределение значений столбца 'lotsize' (топ-10)
print("Распределение значений 'lotsize':")
print(data['lotsize'].str.lower().value_counts().head(10))
# Признак 'lotsize' имеет более 48% пропусков и является числовым. Его использование может ухудшить модель.
# Поэтому удаляем данный признак
data = data.drop('lotsize', axis=1)
print("\nСтолбец 'lotsize' удалён.")

Распределение значений 'lotsize':
lotsize
''               33055
none             28346
'—'              25247
'no data'         5329
'-- sqft lot'     3819
'0.26 acres'      3140
'0.25 acres'      2722
'0.28 acres'      2614
'0.27 acres'      2473
'0.29 acres'      2351
Name: count, dtype: int64

Столбец 'lotsize' удалён.


Признак lotsize содержит большое количество пропусков (пустые строки, 'none', '—', 'no data' и т.д.). Его удаление обосновано низкой информативностью и потенциальными ошибками при заполнении.

## Признак Price/sqft

In [47]:
# Удаляем его для предотвращения утечки.
data = data.drop('Price/sqft', axis=1)
print("Столбец 'Price/sqft' удалён.")

Столбец 'Price/sqft' удалён.


Признак Price/sqft был удалён, поскольку он напрямую коррелирует с целевой переменной, что может привести к утечке данных при обучении модели.

## Обработка признака fireplace

In [48]:
# Приведём значения столбца к нижнему регистру для корректного анализа
data['fireplace'] = data['fireplace'].str.lower()
# Выведем распределение значений (топ-10)
print("Распределение значений 'fireplace' до кодирования:")
print(data['fireplace'].value_counts().head(10))

Распределение значений 'fireplace' до кодирования:
fireplace
yes               71209
1                 14543
2                  2432
not applicable     1993
fireplace           847
3                   564
living room         433
location            399
wood burning        311
gas/gas logs        300
Name: count, dtype: int64


По результатам видно, что значение "yes" встречается 71209 раз, затем идут числовые значения, а также такие записи как "not applicable". Это позволяет понять, что пропуски и некорректные значения могут означать отсутствие камина.

In [49]:
# Определяем набор значений, которые считаются некорректными или обозначают отсутствие камина
false_values = {'', 'no data', 'none', '0', 'not applicable', 'no'}
# Создадим новый столбец 'fireplace_encoded':
# Если значение не является пропуском и не входит в false_values, то считаем, что камин есть (True), иначе – False.
data['fireplace_encoded'] = data['fireplace'].apply(
    lambda x: False if pd.isna(x) or x.strip() in false_values else True
)
# Выведем распределение нового признака
print("\nРаспределение значений 'fireplace_encoded':")
print(data['fireplace_encoded'].value_counts())


Распределение значений 'fireplace_encoded':
fireplace_encoded
False    276485
True     100557
Name: count, dtype: int64


После кодирования булевым значением записи, в которых указана реальная информация о наличии камина, получили значение True, а все пропуски и некорректные записи – False.

In [50]:
# Удаляем исходный столбец 'fireplace' после создания закодированного признака
data.drop('fireplace', axis=1, inplace=True)
print("\nСтолбец 'fireplace' удалён. Текущие столбцы:")
print(data.columns)


Столбец 'fireplace' удалён. Текущие столбцы:
Index(['status', 'baths', 'city', 'schools', 'sqft', 'zipcode', 'beds',
       'state', 'stories', 'target', 'pool_encoded', 'Type', 'Year built',
       'Heating_encoded', 'Cooling_encoded', 'Parking_encoded',
       'fireplace_encoded'],
      dtype='object')


Исходный столбец "fireplace" был удалён, поскольку информация теперь представлена в бинарном виде в столбце "fireplace_encoded". Это упрощает дальнейший анализ и предотвращает утечку данных.

## Обработка признака city

In [51]:
# Приводим названия городов к единому виду – каждый город с заглавной буквы
data['city'] = data['city'].str.title()
# Выведем статистику по пропускам и уникальным значениям
missing_values_count = data['city'].isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100
print("Статистика по столбцу 'city' до очистки:")
print(f"Количество пропущенных значений: {missing_values_count}")
print(f"Процент пропущенных значений: {missing_values_percentage:.2f}%")
print(f"Количество уникальных значений: {data['city'].nunique()}")
print(f"Примеры уникальных значений: {list(data['city'].sort_values().unique()[:10])}")

Статистика по столбцу 'city' до очистки:
Количество пропущенных значений: 32
Процент пропущенных значений: 0.01%
Количество уникальных значений: 1909
Примеры уникальных значений: [' ', '--', 'Abilene', 'Abingdon', 'Accokeek', 'Adams', 'Addison', 'Adel', 'Adelphi', 'Advance']


Мы стандартизировали строковые значения по городам, приведя их к формату Title Case. На данном этапе имеются незначительные пропуски (NaN) и несколько некорректных значений, таких как пустая строка (' ') и '--'.

In [52]:
# Создадим копию DataFrame для работы
data_clean = data.copy()
# Удаляем приставку " City" в названиях городов, если она присутствует
data_clean['city'] = data_clean['city'].str.replace(' City', '', regex=False)
# Сохраняем исходное количество строк
initial_row_count = data_clean.shape[0]
# Удаляем строки, где в столбце 'city' находятся пустые строки, '--' или NaN
filter_mask = ~data_clean['city'].isin([' ', '--']) & data_clean['city'].notna()
data_clean = data_clean[filter_mask]
# Подсчитываем количество удалённых строк
removed_row_count = initial_row_count - data_clean.shape[0]
print(f"Количество удалённых строк: {removed_row_count}")

Количество удалённых строк: 66


После удаления лишней приставки " City" и фильтрации строк с некорректными значениями, удалено 66 строк. Таким образом, остаётся корректное множество городов.

In [53]:
# Выполним финальную проверку пропусков и уникальных значений
missing_values_count = data_clean['city'].isna().sum()
total_count_clean = len(data_clean)
missing_values_percentage = (missing_values_count / total_count_clean) * 100
print("Итоговая статистика по столбцу 'city':")
print(f"Количество пропущенных значений: {missing_values_count}")
print(f"Процент пропущенных значений: {missing_values_percentage:.2f}%")
print(f"Количество уникальных значений: {data_clean['city'].nunique()}")
print(f"Примеры уникальных значений: {list(data_clean['city'].sort_values().unique()[:10])}")

Итоговая статистика по столбцу 'city':
Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 1897
Примеры уникальных значений: ['Abilene', 'Abingdon', 'Accokeek', 'Adams', 'Addison', 'Adel', 'Adelphi', 'Advance', 'Akron', 'Alamo Heights']


После очистки в столбце "city" отсутствуют пропуски, количество уникальных значений уменьшилось до 1897, а первые 10 уникальных значений выглядят корректно. Теперь данные готовы для дальнейшего анализа.

## Обработаем признак schools

In [54]:
# Просмотр примера строки в столбце 'schools'
print("Пример строки в 'schools':")
print(data['schools'].iloc[5])

Пример строки в 'schools':
[{'rating': [], 'data': {'Distance': [], 'Grades': []}, 'name': []}]


Например, одна из строк выглядит так:
```python 
"[{'rating': [], 'data': {'Distance': [], 'Grades': []}, 'name': []}]"
```
Это позволяет понять, что внутри содержатся сведения о рейтинге и расстоянии, а неструктурированное представление затрудняет дальнейшую обработку.

In [55]:
# Рассчитаем средний рейтинг
def process_rating(rating):
    rating = rating[0].replace('/10', '')
    extracted_numbers = [float(num) for num in rating if num.isdigit()]
    average_rating = np.average(extracted_numbers) if extracted_numbers else -1
    return round(average_rating, 1)

s_rating = (
    data.schools.str.findall(r"\brating': ([\s\S]+?), 'data\b")
    .apply(lambda x: process_rating(x))
)

# Создадим признак school_rating _mean в нашем наборе данных. Пропуски заполним -1 
data['school_rating_mean'] = s_rating

In [56]:
#Расчитаем минимальную дистанцию
s_distance = data.schools.str.findall(r"\bDistance': ([\s\S]+?), 'Grades\b") 
# Извлечение расстояний до школ из каждой строки
s_distance = s_distance.apply(lambda x: x[0]) 
# Получение первого найденного значения списков, полученных в результате предыдущего шага
s_distance = s_distance.str.replace('[a-zA-Z]','', regex=True) 
# Удаление всех символов алфавита из расстояний
s_distance = s_distance.str.findall(r'\b([0-9]+.[0-9]+)') 
# Извлечение чисел виде Точноечисло, включая десятичные значения
s_distance = s_distance.apply(lambda x: [float(i) for i in x]) 
# Конвертирование каждого числового значения списка в тип float
school_dist_min = s_distance.apply(lambda x: -1 if len(x)==0 else min(x)) 
# Нахождение минимального значения в каждом списке s_distance, и возвращение -1, если длина списка равна 0

#Создадим признак school_dist_min в нашем наборе данных. Пропуски заполним -1
data['school_dist_min'] = school_dist_min

In [57]:
# теперь признак schools можно удалить
data.drop('schools', axis=1, inplace=True)

Исходный столбец «schools» был удалён, поскольку необходимая информация теперь представлена в признаках school_rating_mean и school_dist_min

## Обработаем признак sqft (площадь)

In [58]:
# выполним проверку
missing_values_count = data['sqft'].isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
f"Количество уникальных значений: {data['sqft'].nunique()}\n"
f"Уникальные значения: {list(data['sqft'].sort_values().unique()[:10])}")

Количество пропущенных значений: 40535
Процент пропущенных значений: 10.75%
Количество уникальных значений: 25405
Уникальные значения: ['--', '-- sqft', '0', '1', '1 sqft', '1,000', '1,000 sqft', '1,001', '1,001 sqft', '1,002']


In [59]:
# Обрабатываем непустые значения столбца sqft: выделяем все цифровые последовательности
sqft_non_missing = data.loc[data['sqft'].notna(), 'sqft']
sqft_clean = sqft_non_missing.str.findall(r'\d+').apply(lambda x: ''.join(x))
# Обновляем значения sqft для тех строк, где значение не пропущено
data.loc[data['sqft'].notna(), 'sqft'] = sqft_clean
# Заменяем оставшиеся пропуски значений на "0"
data['sqft'] = data['sqft'].fillna(0)
# Если встречаются пустые строки, заменяем их тоже на 0
data.loc[data['sqft'] == '', 'sqft'] = 0
# Приводим столбец к целочисленному типу
data['sqft'] = data['sqft'].astype(int)

In [60]:
# Строки, где sqft равен 0 и тип недвижимости входит в список ['other', 'land', 'ranch']
selected_rows = data[(data['sqft'] == 0) & (data['Type'].isin(['other', 'land', 'ranch']))]
# Просмотр первых нескольких строк
display(selected_rows.head())
# Вычисление процента от общего числа строк
percent_selected = len(selected_rows) / total_count * 100
print(f"Выбранных строк: {len(selected_rows)}")
print(f"Процент выбранных строк от общего числа строк: {percent_selected:.2f}%")

,status,baths,city,sqft,zipcode,beds,state,stories,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating_mean,school_dist_min
4,For Sale,0.0,Palm Bay,0,32908,NaN,FL,NaN,"$5,000",False,land,no date,False,False,False,False,4.7,3.03
7,Active,0.0,Memphis,0,38115,NaN,TN,NaN,"68,000",False,other,1976,False,False,False,False,2.7,0.40
26,Active,1.0,Miami,0,33183,Baths,FL,NaN,"799,000",False,other,no date,False,False,False,False,3.8,0.90
40,Active,0.0,Rotonda West,0,33947,NaN,FL,NaN,"54,000",False,other,no date,False,False,False,False,6.3,3.10
41,For Sale,0.0,San Diego,0,92127,NaN,CA,NaN,"$1,100,000",False,land,no date,False,False,False,False,9.0,0.97


Выбранных строк: 46186
Процент выбранных строк от общего числа строк: 12.25%


In [61]:
# Ячейка 5: Строки, где sqft равен 0, но тип недвижимости не входит в список ['other', 'land', 'ranch']
selected_rows_diff = data[(data['sqft'] == 0) & ~(data['Type'].isin(['other', 'land', 'ranch']))]
# Просмотр первых нескольких строк
display(selected_rows_diff.head())
# Вычисление уникальных значений признака Type и процента выборки
unique_types = selected_rows_diff['Type'].unique()
percent_selected_diff = len(selected_rows_diff) / total_count * 100
print(f"Уникальные значения признака Type в выбранных строках: {unique_types}")
print(f"Выбранных строк: {len(selected_rows_diff)}")
print(f"Процент выбранных строк от общего числа строк: {percent_selected_diff:.2f}%")

,status,baths,city,sqft,zipcode,beds,state,stories,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating_mean,school_dist_min
15,For Sale,0.0,Brooklyn,0,11219,NaN,NY,NaN,"1,650,000",False,single_family_home,1905,False,True,False,False,6.3,0.20
56,Active,2.0,Southern Pines,0,28387,2,NC,1,"$166,500",False,condo,1998,False,False,False,False,5.6,0.90
193,Foreclosure,0.0,Pottstown,0,19464,4 bd,PA,NaN,"$170,933",False,single_family_home,1974,True,False,False,False,4.7,2.20
221,For Sale,8.0,New York,0,10075,6 Beds,NY,NaN,"$15,000,000",False,condo,1930,False,False,False,False,4.3,0.19
232,For Sale,2.0,Chicago,0,60654,2 Beds,IL,2.0,"$525,000",False,condo,2003,True,True,True,False,4.0,0.57


Уникальные значения признака Type в выбранных строках: ['single_family_home' 'condo' 'miscellaneous' 'multi_family_home'
 'townhouse' 'mobile_home' 'modern' 'apartment']
Выбранных строк: 6987
Процент выбранных строк от общего числа строк: 1.85%


In [62]:
# Ячейка 6: Вывод заключительного сообщения
print("Большая часть строк с нулевыми значениями в sqft относится к типам 'other', 'land' и 'ranch'.")
print("Оставшиеся 1.85% строк с нулевым значением sqft относятся к другим типам недвижимости, что свидетельствует о корректности заполнения данными.")

Большая часть строк с нулевыми значениями в sqft относится к типам 'other', 'land' и 'ranch'.
Оставшиеся 1.85% строк с нулевым значением sqft относятся к другим типам недвижимости, что свидетельствует о корректности заполнения данными.


## Обработаем признак zipcode

In [63]:
# Первичный анализ столбца 'zipcode'
missing_values_count = data['zipcode'].isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100
print("Статистика по признаку 'zipcode' до обработки:")
print(f"Количество пропущенных значений: {missing_values_count}")
print(f"Процент пропущенных значений: {missing_values_percentage:.2f}%")
print(f"Количество уникальных значений: {data['zipcode'].nunique()}")
print(f"Примеры уникальных значений: {list(data['zipcode'].sort_values().unique()[:10])}")

Статистика по признаку 'zipcode' до обработки:
Количество пропущенных значений: 0
Процент пропущенных значений: 0.00%
Количество уникальных значений: 4549
Примеры уникальных значений: ['--', '0', '00000', '02108', '02109', '02110', '02111', '02113', '02114', '02115']


На данном этапе отсутствуют пропуски, но наблюдается большое количество уникальных значений (4549), среди которых встречаются некорректные значения, такие как '--', '0' и '00000'.

In [64]:
# Отфильтровываем строки с некорректными значениями zipcode: '--', '0' и '00000'
indexes_to_drop = data[(data['zipcode'] == '--') | 
                       (data['zipcode'] == '0') | 
                       (data['zipcode'] == '00000')].index
# Удаляем отобранные строки
data = data.drop(indexes_to_drop)
# Проверка после удаления некорректных строк
print("Некорректные значения удалены. Примеры zipcode после удаления:")
print(list(data['zipcode'].sort_values().unique()[:10]))

Некорректные значения удалены. Примеры zipcode после удаления:
['02108', '02109', '02110', '02111', '02113', '02114', '02115', '02116', '02118', '02119']


После удаления строк с некорректными значениями количество строк уменьшилось, а первые 10 значений zipcode выглядят корректно (например, '02108', '02109', …).

In [65]:
# Если в zipcode встречается расширенный формат (через дефис), оставляем только первую часть до дефиса.
data['zipcode'] = data['zipcode'].str.replace(r"-.+", "", regex=True)
# Проверка преобразованных значений
print("Примеры zipcode после корректировки:")
print(list(data['zipcode'].sort_values().unique()[:10]))

Примеры zipcode после корректировки:
['02108', '02109', '02110', '02111', '02113', '02114', '02115', '02116', '02118', '02119']


После применения регулярного выражения все значения zipcode теперь содержат только первую часть, что соответствует корректному формату почтовых индексов.

## Обработаем признак bads(количество спален)

In [66]:
# Анализ признака 'beds'
missing_values_count = data['beds'].isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100
print("Статистика по признаку 'beds' до удаления:")
print(f"Количество пропущенных значений: {missing_values_count}")
print(f"Процент пропущенных значений: {missing_values_percentage:.2f}%")
print(f"Количество уникальных значений: {data['beds'].nunique()}")
print(f"Уникальные значения: {list(data['beds'].sort_values().unique()[:10])}")
# Признак 'beds' содержит неоднозначные данные (часть строк — текст, часть — числовые значения, часть — площади)
# Поэтому столбец 'beds' удаляем
data = data.drop('beds', axis=1)
print("\nСтолбец 'beds' успешно удалён.")

Статистика по признаку 'beds' до удаления:
Количество пропущенных значений: 91200
Процент пропущенных значений: 24.19%
Количество уникальных значений: 1183
Уникальные значения: [' ', '# Bedrooms 1st Floor', '-- bd', '-- sqft', '0', '0.0', '0.25 acres', '0.26 acres', '0.27 acres', '0.28 acres']

Столбец 'beds' успешно удалён.


Признак beds имеет 24.18% пропусков и большое количество уникальных значений с различными форматами. Такие данные сложно интерпретировать, поэтому столбец удаляется.

## Обработаем признак state

In [67]:
print("Статистика по признаку 'state':")
print(f"Количество пропущенных значений: {data['state'].isna().sum()}")
print(f"Количество уникальных значений: {data['state'].nunique()}")
print(f"Уникальные значения: {list(data['state'].unique()[:10])}")

Статистика по признаку 'state':
Количество пропущенных значений: 0
Количество уникальных значений: 38
Уникальные значения: ['NC', 'WA', 'CA', 'TX', 'FL', 'PA', 'TN', 'IA', 'NY', 'OR']


Признак state не содержит пропусков, имеет 37 уникальных значений (например, 'NC', 'WA', 'CA', 'TX', 'FL', 'PA', 'TN', 'IA', 'NY', 'OR') и выглядит корректно для дальнейшего анализа.

## обработаем признак stories

In [68]:
# Проверяем количество пропусков, процент пропусков, количество уникальных значений и примеры уникальных значений
missing_values_count = data['stories'].isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100
print("Статистика по признаку 'stories':")
print(f"Количество пропущенных значений: {missing_values_count}")
print(f"Процент пропущенных значений: {missing_values_percentage:.2f}%")
print(f"Количество уникальных значений: {data['stories'].nunique()}")
print(f"Примеры уникальных значений: {list(data['stories'].sort_values().unique()[:10])}")

Статистика по признаку 'stories':
Количество пропущенных значений: 150576
Процент пропущенных значений: 39.94%
Количество уникальных значений: 347
Примеры уникальных значений: [', 1', ', 2', ', 3', '0', '0.0', '1', '1 1/2 Levels', '1 1/2 Story', '1 Level', '1 Level, 1.5 Level']


Признак "stories" имеет высокую долю пропусков (примерно 39.93%), а количество уникальных значений составляет 346 (например, [', 1', ', 2', ', 3', '0', '0.0', '1', '1 1/2 Levels', '1 1/2 Story', '1 Level', '1 Level, 1.5 Level']).

In [69]:
# Анализируем, в каких типах объектов встречаются пропуски в признаке "stories"
missing_stories_by_type = data[data['stories'].isna()]['Type'].value_counts()
print("Распределение типов объектов среди строк с пропущенным значением 'stories':")
print(missing_stories_by_type)

Распределение типов объектов среди строк с пропущенным значением 'stories':
Type
single_family_home    46304
other                 37032
land                  28838
condo                 24356
multi_family_home      5212
townhouse              3938
mobile_home            1499
ranch                  1389
modern                 1129
apartment               824
historical               44
miscellaneous            11
Name: count, dtype: int64


В пропусках встречаются как типы "земля" (land), так и домов (например, single_family_home, condo и т.д.). Пропуски присутствуют в значительном числе строк, поэтому заполнять их случайными значениями нецелесообразно.

In [70]:
# Удаляем признак "stories", так как он содержит большое количество пропусков в разных типах объектов
data = data.drop('stories', axis=1)
print("Столбец 'stories' успешно удалён. Текущие столбцы:")
print(data.columns)

Столбец 'stories' успешно удалён. Текущие столбцы:
Index(['status', 'baths', 'city', 'sqft', 'zipcode', 'state', 'target',
       'pool_encoded', 'Type', 'Year built', 'Heating_encoded',
       'Cooling_encoded', 'Parking_encoded', 'fireplace_encoded',
       'school_rating_mean', 'school_dist_min'],
      dtype='object')


Признак "stories" был удалён, поскольку его заполнение не представляется корректным и информативным из-за большого количества пропусков в различных типах недвижимости.

## Посмотрим на целевой признак target

In [71]:
# Первичная проверка столбца 'target'
missing_values_count = data['target'].isna().sum()
total_count = len(data)
missing_values_percentage = (missing_values_count / total_count) * 100
print("Статистика по столбцу 'target' до обработки:")
print(f"Количество пропущенных значений: {missing_values_count}")
print(f"Процент пропущенных значений: {missing_values_percentage:.2f}%")
print(f"Количество уникальных значений: {data['target'].nunique()}")
print(f"Примеры уникальных значений: {list(data['target'].sort_values().unique()[:10])}")

Статистика по столбцу 'target' до обработки:
Количество пропущенных значений: 2480
Процент пропущенных значений: 0.66%
Количество уникальных значений: 43938
Примеры уникальных значений: ['$1', '$1,000', '$1,000,000', '$1,000,000+', '$1,000,036', '$1,000,050', '$1,000,100', '$1,000,898', '$1,000/mo', '$1,001,713']


Изначально обнаружено 2475 пропусков (0.66%), 43936 уникальных значений, среди которых встречаются строки с символами '$', запятыми и текстовыми пометками (например, "$1,000/mo").

In [72]:
# Удаляем строки с пропущенными значениями в столбце 'target'
data = data.dropna(subset=['target'])
print("Строки с пропущенным значением 'target' удалены.")

Строки с пропущенным значением 'target' удалены.


Удалены все строки с пропущенными значениями в target, так как их заполнить корректно невозможно.

In [73]:
# Определяем строки, где в target встречается '/mo'
rental_rows = data[data['target'].str.contains('/mo', regex=True)]
percent_rentals = (len(rental_rows) / len(data)) * 100
print(f"{round(percent_rentals, 2)}% строк содержат '/mo' в столбце 'target'.")
# Удаляем строки с обозначением аренды
data = data[~data['target'].str.contains('/mo', regex=True)]
print("Строки с арендными значениями (с '/mo') удалены.")

0.11% строк содержат '/mo' в столбце 'target'.
Строки с арендными значениями (с '/mo') удалены.


Лишь 0.11% значений относятся к аренде. Такие строки удаляются, т.к. задача – прогноз стоимости недвижимости, а не аренды.

In [74]:
# Удаляем все символы, кроме цифр, из столбца target
data['target'] = data['target'].str.replace('[^0-9]', '', regex=True)
# Преобразуем значения в целочисленный тип
data['target'] = data['target'].astype(int)
print("Значения в 'target' преобразованы в числовой формат.")
# Выводим минимальное и максимальное значения для проверки
print("Минимальное значение target:")
display(data[data['target'] == data['target'].min()].head())
print("Максимальное значение target:")
display(data[data['target'] == data['target'].max()].head())

Значения в 'target' преобразованы в числовой формат.
Минимальное значение target:


,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating_mean,school_dist_min
15533,Active,2.0,Goodlettsville,1596,37072,TN,1,False,single_family_home,1940,True,False,True,False,2.3,0.2
25334,Other,1.0,Indianapolis,1586,46224,IN,1,False,single_family_home,1957,True,True,True,True,1.5,0.2
84943,Foreclosure,0.0,Cincinnati,2536,45231,OH,1,False,land,1865,False,True,True,True,2.0,0.5
148219,Active,0.0,Huntley,0,60142,IL,1,False,land,no date,False,False,False,False,-1.0,-1.0
205315,Active,0.0,Huntley,0,60142,IL,1,False,land,no date,False,False,False,False,7.8,0.9


Максимальное значение target:


,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating_mean,school_dist_min
132407,For Sale,18.0,Los Angeles,0,90077,CA,195000000,True,single_family_home,1935,True,True,True,True,2.7,1.3


После удаления лишних символов получены чистые числовые значения. Минимальное и максимальное значения можно просмотреть для оценки распределения.

In [75]:
# Подсчитываем количество строк, где target менее 1000
anomaly_count = len(data[data['target'] < 1000])
anomaly_percentage = (anomaly_count / len(data)) * 100
print(f"Количество строк с target < 1000: {anomaly_count}")
print(f"Процент таких строк: {anomaly_percentage:.2f}%")
# Удаляем аномальные строки
data = data.drop(data[data['target'] < 1000].index)
print("Строки с target < 1000 удалены.")
# Проверяем минимальное значение после удаления
print("Минимальное значение target после удаления аномалий:")
display(data[data['target'] == data['target'].min()].head())

Количество строк с target < 1000: 77
Процент таких строк: 0.02%
Строки с target < 1000 удалены.
Минимальное значение target после удаления аномалий:


,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating_mean,school_dist_min
1357,Foreclosure,2.0,Detroit,2544,48206,MI,1000,False,single_family_home,1914,True,False,True,False,3.2,0.79
1682,Other,2.0,Detroit,1572,48219,MI,1000,False,single_family_home,1940,True,False,True,False,1.0,0.70
2282,Foreclosure,2.0,Detroit,2048,48205,MI,1000,False,single_family_home,1925,True,False,True,False,2.9,1.08
2644,Foreclosure,2.0,Detroit,1827,48227,MI,1000,False,single_family_home,1928,True,False,True,False,2.9,0.56
2673,Active,0.0,Memphis,0,38106,TN,1000,False,other,no date,False,False,False,False,2.8,0.60


Строк с отклонением (target < 1000) оказалось всего 77 (0.02% от общего числа). Такие строки удалены.

In [76]:
# Фильтрация строк, где значение 'baths' равно 0,
# 'sqft' равно 0 и 'Year built' имеет значение 'no date'
rows_without_useful_info = data[
    (data['baths'] == 0) &
    (data['sqft'] == 0) &
    (data['Year built'] == 'no date')
]
# Вывод первых 5 строк для проверки
display(rows_without_useful_info.head())
# Подсчёт строк с некорректными данными и вычисление процента
num_rows_invalid = len(rows_without_useful_info)
total_rows = len(data)
percentage_invalid = num_rows_invalid / total_rows * 100
print(f"Найдено {num_rows_invalid} строк с некорректными данными из {total_rows} общего числа строк "
      f"({percentage_invalid:.2f}%).")

,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating_mean,school_dist_min
4,For Sale,0.0,Palm Bay,0,32908,FL,5000,False,land,no date,False,False,False,False,4.7,3.03
40,Active,0.0,Rotonda West,0,33947,FL,54000,False,other,no date,False,False,False,False,6.3,3.10
41,For Sale,0.0,San Diego,0,92127,CA,1100000,False,land,no date,False,False,False,False,9.0,0.97
42,Active,0.0,New York,0,10006,NY,850000,False,other,no date,False,False,False,False,7.0,0.10
48,Active,0.0,Signal Mountain,0,37377,TN,125000,False,other,no date,False,False,False,False,6.3,3.90


Найдено 42012 строк с некорректными данными из 374081 общего числа строк (11.23%).


In [77]:
# Удаление строк, не содержащих полезной информации для прогноза
data = data[
    ~((data['baths'] == 0) &
      (data['sqft'] == 0) &
      (data['Year built'] == 'no date'))
]
# Вывод первых строк очищенного DataFrame
display(data.head())
# Получение общей информации о DataFrame
data.info()

,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating_mean,school_dist_min
0,Active,4.0,Southern Pines,2900,28387,NC,418000,False,single_family_home,2019,True,False,False,True,5.2,2.70
1,For Sale,3.0,Spokane Valley,1947,99216,WA,310000,False,single_family_home,2019,False,False,False,False,4.0,1.01
2,For Sale,2.0,Los Angeles,3000,90049,CA,2895000,True,single_family_home,1961,True,True,True,True,6.7,1.19
3,For Sale,8.0,Dallas,6457,75205,TX,2395000,False,single_family_home,2006,True,True,True,True,5.6,0.10
5,For Sale,0.0,Philadelphia,897,19145,PA,209000,False,townhouse,1920,True,True,False,False,-1.0,-1.00


<class 'pandas.core.frame.DataFrame'>
Index: 332069 entries, 0 to 377043
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   status              332069 non-null  object 
 1   baths               332069 non-null  float64
 2   city                332060 non-null  object 
 3   sqft                332069 non-null  int32  
 4   zipcode             332069 non-null  object 
 5   state               332069 non-null  object 
 6   target              332069 non-null  int32  
 7   pool_encoded        332069 non-null  bool   
 8   Type                332069 non-null  object 
 9   Year built          332069 non-null  object 
 10  Heating_encoded     332069 non-null  bool   
 11  Cooling_encoded     332069 non-null  bool   
 12  Parking_encoded     332069 non-null  bool   
 13  fireplace_encoded   332069 non-null  bool   
 14  school_rating_mean  332069 non-null  float64
 15  school_dist_min     332069 non-null  fl

In [78]:
# Сохранение предобработанных данных в CSV файл без записи индексов
data.to_csv("data/cleaned_data.csv", index=False)
print("Данные успешно сохранены в 'data/cleaned_data.csv'.")

Данные успешно сохранены в 'data/cleaned_data.csv'.
